# Pauli Objects

## Introduction and Arithmetic

Pauli operators acting on the $N$-qubit Hilbert space are ubiquitous in quantum circuit computations. In domain applications like QAOA and VQE, these often appear with a sparse structure corresponding to the physics of the problem at hand. To efficiently manipulate such operators and compute expectation values over the relevant Pauli strings, `quasar` features a `Pauli` class (and associated utility classes) that provide for sparse representation and intuitive manipulation of Pauli-type objects. 

To get started, let's grab some utility objects that generate the Pauli operators on individual qubit indices:

In [1]:
from qcware import forge
# this line is for internal tracking; it is not necessary for use!
forge.config.set_environment_source_file('pauli_operators.ipynb')

import numpy as np
import quasar
I, X, Y, Z = quasar.Pauli.IXYZ()

Now, we can use these to start building up Pauli operators. 

In [2]:
pauli = I[-1] + X[0] - X[0] * Y[1] + 0.5 * X[0] * Y[1] * Z[2] - X[0] * Y[1] * Z[2] / 4.0
pauli += 1.0
pauli += 2.0 * (X[0] + Z[0]) * (X[0] + Y[0])
pauli += pauli
pauli /= 2.0
print(pauli)

+4.0*I
+(1-2j)*X0
-1.0*X0*Y1
+0.25*X0*Y1*Z2
+2j*Y0
+2j*Z0


How did that work? Next, `I/X/Y/Z` are "`PauliStarter`" objects that generate `Pauli` objects when indexed by the `[]` operator:

In [3]:
print(type(X))
print(X[3])
print(type(X[3]))

<class 'quasar.pauli.PauliStarter'>
+1.0*X3
<class 'quasar.pauli.Pauli'>


**Technical Note:** Note that in the `Pauli` objects themselves, we only store the overall `I` operator and products of `X`, `Y`, and `Z` strings. Therefore, the `I` `PauliStarter` object is indexed the same as the `X`, `Y`, and `Z` `PauliStarter` objects, but the specific index applied to the `I` `PauliStarter` object has no effect. A loose (but not required convention) is to use `I[-1]` to indicate the overall `I` operator:

In [4]:
print(I[-1])
print(I[1])
print(I[-1] * X[0])
print(I[1] * X[0])

+1.0*I
+1.0*I
+1.0*X0
+1.0*X0


Next, we have overloaded all relevant arithmetic operators within the `Pauli` class to allow arithmetic composition of linear combinations of products of single-qubit Pauli operators. These work with either `float` or `complex` coefficients:

In [5]:
print(-1.j * X[0])

-1j*X0


Note that products of Pauli operators on the same qubit index are automatically reduced to linear or constant complexity by the Pauli multiplication rules:

In [6]:
print(X[0]*X[0])
print(X[0]*Y[0])
print(X[0]*Z[0])
print(Y[0]*X[0])
print(Y[0]*Y[0])
print(Y[0]*Z[0])
print(Z[0]*X[0])
print(Z[0]*Y[0])
print(Z[0]*Z[0])
print(X[0]*Y[0]*Z[0])
print(Y[0]*X[0]*Z[0])

+1.0*I
+1j*Z0
-1j*Y0
-1j*Z0
+1.0*I
+1j*X0
+1j*Y0
-1j*X0
+1.0*I
+1j*I
-1j*I


## Pauli Summary Attributes

The unique occupied qubit indices over all strings encountered in a `Pauli` object are provided in the `qubits` property:

In [7]:
pauli = X[0] + Y[2] + Z[3]
print(pauli.qubits)

SortedSet([0, 2, 3])


The attribute `nqubit` is the total number of qubits between `min_qubit` and `max_qubit` (including any blank qubit indices). The attribute `nqubit_sparse` is the number of occupied qubits (excluding bland qubit indices):

In [8]:
print(pauli.nqubit)
print(pauli.nqubit_sparse)
print(pauli.min_qubit)
print(pauli.max_qubit)

4
3
0
3


The maximum order attribute is the maximum number of one-qubit Pauli operators currently present in the `Pauli` object:

In [9]:
print(pauli.max_order)

1


The number of terms attribute is the number of Pauli strings currently present in the `Pauli` object:

In [10]:
print(pauli.nterm)

3


These are easily printable in a handy summary string:

In [11]:
print(pauli.summary_str)

Pauli:
  nqubit     = 4
  nterm      = 3
  max_order  = 1



## Dictionary Structure

The `Pauli` class actually inherits from the `collections.OrderedDict` class, and internally is represented as an `OrderedDict` of `PauliString : float/complex` pairs.

In [12]:
pauli = X[0] + X[0] * Y[1]
for string, value in pauli.items():
    print('%-8s : %6s' % (string, value))

X0       :    1.0
X0*Y1    :    1.0


We can use `dict` access methods to access and modify the coefficients of existing Pauli strings:

In [13]:
pauli[quasar.PauliString.from_string('X0*Y1')] += 1.0
print(pauli)

+1.0*X0
+2.0*X0*Y1


and even add new Pauli strings:

In [14]:
pauli[quasar.PauliString.from_string('X0*Y4')] = 1.0
print(pauli)

+1.0*X0
+2.0*X0*Y1
+1.0*X0*Y4


All `dict` access methods have been overloaded to also accept `str` arguments, which will be converted to `PauliString` under the hood:

In [15]:
pauli['X0'] += 1.0
print(pauli['X0'])
print(pauli['X0*Y1'])

2.0
2.0


## PauliString and PauliOperator Utility Classes

Utility class `PauliString` represents a string of one or more `PauliOperator` objects:

In [16]:
string = quasar.PauliString.from_string('X0*Y1*Z2')
print(string)
print(string.order)
print(string.qubits)
print(string.chars)

X0*Y1*Z2
3
(0, 1, 2)
('X', 'Y', 'Z')


From a data structure perspective, `PauliString` inherits from `tuple` and contains a `tuple` of `PauliOperator` objects:

In [17]:
print(string[0])
print(type(string[0]))

X0
<class 'quasar.pauli.PauliOperator'>


Utility class `PauliOperator` represents a one-qubit Pauli operator with a given character and qubit index:

In [18]:
operator = quasar.PauliOperator.from_string('Y10')
print(operator)
print(operator.qubit)
print(operator.char)

Y10
10
Y


## Symmetry Considerations

In practical problems involving two-qubit and $k$-qubit interactions, one usually encounters one of two possible conventions in the literature/code structure: the upper triangular representation or the symmetric representation. To permit both possibilities, `quasar` considers, e.g., `Z0*Z1` and `Z1*Z0` to be inequivalent `PauliString` objects:

In [19]:
print(Z[0]*Z[1] + Z[1]*Z[0])

+1.0*Z0*Z1
+1.0*Z1*Z0


In [20]:
print(Z[0]*Z[1] + Z[0]*Z[1])

+2.0*Z0*Z1


## Additional Math Functions

A few additional math functions are provided in `Pauli` for convenience/advanced users. The `conj` property returns a new version of the `Pauli` object with the values conjugated:

In [21]:
print(pauli.conj)

+2.0*X0
+2.0*X0*Y1
+1.0*X0*Y4


The `dot` method takes the dot product of coincident Pauli strings in two `Pauli` objects:

In [22]:
pauli = 3.0 * I[-1] + 4.0 * Z[0] * Z[1]
print(pauli.dot(pauli))

25.0


The `norm2` property returns the square root of the dot product of `self` with `self`:

In [23]:
print(pauli.norm2)

5.0


The `norminf` property returns the magnitude of the largest coefficient of `self`:

In [24]:
print(pauli.norminf)

4.0


The `zero` static method returns a new `Pauli` object initialized with no strings:

In [25]:
print(quasar.Pauli.zero())

The `zeros_like` static method returns a new `Pauli` object initialized with the strings of another `Pauli` object, but with coefficient values initialized to zero:

In [26]:
print(quasar.Pauli.zeros_like(pauli))

+0.0*I
+0.0*Z0*Z1


Many times, arithmetic manipulations yield Pauli strings with small or even zero coefficient values. A new `Pauli` object with such strings removed can be obtained by calling the `sieved` method, which takes an optional `cutoff` argument:

In [27]:
pauli -= 3.0 * I[-1]
print(pauli)

+0.0*I
+4.0*Z0*Z1


In [28]:
print(pauli.sieved())

+4.0*Z0*Z1


In [29]:
print(pauli.sieved(cutoff=1.0E-14))

+4.0*Z0*Z1


## Hilbert Space Representation

Sometimes, it can be quite useful to see what a `Pauli` object looks like in the computational-basis Hilbert space, or to see how the `Pauli` object acts on a statevector in the computational-basis Hilbert space. 

In [30]:
pauli = X[0] * X[1]
print(pauli.to_matrix())
print(pauli.matrix_vector_product(np.array([1, 2, 3, 4])))

[[0.+0.j 0.+0.j 0.+0.j 1.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [1.+0.j 0.+0.j 0.+0.j 0.+0.j]]
[4.+0.j 3.+0.j 2.+0.j 1.+0.j]


It can also be very useful to convert from a computational-basis Hilbert space operator to an equivalent `Pauli` form:

In [31]:
matrix = quasar.Matrix.YY
print(matrix)
pauli = quasar.Pauli.from_matrix(matrix)
print(pauli)

[[ 0.+0.j  0.-0.j  0.-0.j -1.+0.j]
 [ 0.+0.j  0.+0.j  1.-0.j  0.-0.j]
 [ 0.+0.j  1.-0.j  0.+0.j  0.-0.j]
 [-1.+0.j  0.+0.j  0.+0.j  0.+0.j]]
+(1+0j)*Y0*Y1
